In [4]:
!pip install "openai==0.28.1" "jsonformer" "transformers"

You should consider upgrading via the 'C:\Projects\MimicBot\mimicbotwrapper\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
from jsonformer import Jsonformer
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-3b")
tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b")

json_schema = {
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
        },
        "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
        },
        "cost": {
            "type": "number",
            "minimum": 0
        }
    }
}

prompt = "Generate a person's information based on the following schema:"
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt)
generated_data = jsonformer()

print(generated_data)

In [7]:
# DOCS https://platform.openai.com/docs/guides/gpt/function-calling
import os
import openai
openai.api_key = None

In [ ]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "Hi!"},
    ],
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]
                    },
                    "cost": {
                        "type": "number",
                        "minimum": 0
                    }
                },
                "required": ["location"]
            }
        }
    ],
    function_call="auto"
)
res.choices[0]

In [11]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "What is the weather in Boston?"},
    ],
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]
                    },
                    "cost": {
                        "type": "number",
                        "minimum": 0
                    }
                },
                "required": ["location"]
            }
        }
    ],
    function_call="auto"
)
res.choices[0]

<OpenAIObject at 0x20ab53e02c0> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "get_current_weather",
      "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
    }
  },
  "finish_reason": "function_call"
}

In [14]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "What is the weather in Boston?"},
        {
            "role": "assistant",
            "content": None,
            "function_call":
                {
                    "name": "get_current_weather",
                    "arguments": "{ \"location\": \"Boston, MA\"}"
                }
        },
        {
            "role": "function",
            "name": "get_current_weather",
            "content": "{\"temperature\": 22, \"unit\": \"celsius\", \"description\": \"Sunny\"}"
        }
    ],
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]
                    },
                    "cost": {
                        "type": "number",
                        "minimum": 0
                    }
                },
                "required": ["location"]
            }
        }
    ],
    function_call="auto"
)
res.choices[0]

<OpenAIObject at 0x20ab539d800> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": "The current weather in Boston, MA is sunny with a temperature of 22 degrees Celsius."
  },
  "finish_reason": "stop"
}